In [ ]:
library(dplyr)

In [ ]:
statistical_table <- data.frame(p=seq(.1,.9,by=.1),
                                Prior=c(1,1,1,1,1,2,2,1,1))
statistical_table

In [ ]:
statistical_table %>% mutate( Prior = Prior / sum(Prior)) -> statistical_table
statistical_table

In [ ]:
statistical_table$Likelihood <- dbinom(12, size=20, prob = statistical_table$p)
statistical_table

In [ ]:
statistical_table$Product <- statistical_table$Prior * statistical_table$Likelihood
statistical_table

In [ ]:
statistical_table$Posterior <- statistical_table$Product / sum(statistical_table$Prior * statistical_table$Likelihood)
statistical_table

In [ ]:
par(mfrow=c(1,3))

barplot(statistical_table$Prior,
        main="Prior distribution",
        xlab="Parameter values",
        ylab="Prior probability",
        names.arg=statistical_table$p,
        col="darkred",
        horiz=FALSE)

barplot(statistical_table$Likelihood,
        main="Likelihood distribution",
        xlab="Parameter values",
        ylab="Likelihood",
        names.arg=statistical_table$p,
        col="darkred",
        horiz=FALSE)

barplot(statistical_table$Posterior,
        main="Posterior distribution",
        xlab="Parameter values",
        ylab="Posterior probability",
        names.arg=statistical_table$p,
        col="darkred",
        horiz=FALSE)

Sampling from a posterior distribution:

Markov chain monte carlo metropolis algorithm



---

**Markov chain**

Monte Carlo

Metropolis algorithm

There are certain properties worth mentioning here with respect to a markov chain:

**Irreducibility**: Given the Markov chain, the organism can transition from any given state to any other given state in one or more steps.

**Periodicity**: The organism present in a given state can only come back to the given state in regular intervals. However, over here because the organism can undergo reverse metamorphosis in any direction, the Markov chain in our example is aperiodic.

**Unique stationary distribution**: If a Markov chain is irreducible and aperiodic, as is the case here, it will have a unique stationary distribution i.e **as one takes infinite number of steps, the probability of transitioning to a given state does not depend on the initial state**.

In [ ]:
p <- c(0, 1, 0, 0, 0)

In [ ]:
P <- matrix(c(.5, .5, 0, 0, 0,
              .25, .5, .25, 0, 0,
              0, .25, .5, .25, 0,
              0, 0, .25, .5, .25,
              0, 0, 0, .5, .5),
            nrow=5, ncol=5, byrow=TRUE)
P

In [ ]:
print(p %*% P, digits = 5)

In [ ]:
print(p %*% P %*% P %*% P %*% P, digits = 5)

In [ ]:
Pm <- diag(rep(1, 5))

In [ ]:
for(j in 1:100){
  Pm <- Pm %*% P
}
print(Pm, digits = 5)

Sampling from a posterior distribution:

Markov chain monte carlo metropolis algorithm


---


Markov chain

**Monte Carlo**

**Metropolis algorithm**

Metropolis algorithm allows one to simulate from the above probability distribution using a simple random walk:

a) We start from any state the organism can be in, A to E.

b) To decide which state the organism would take next, a fair coin is flipped. Let us say, if the coin flip lands up heads, we move one state to the left. If it lands up tails, we move one state to the right. This is called the candidate state.

c) Next we calculate the ratio of the probabilities of the candidate state and the current state.

    R = probability(candidate state) / probability(current state)

d) Next we choose a number N between 0 and 1 at random. If N is smaller than R, we move to the candidate state, otherwise we remain at the current state.

In [ ]:
pd <- function(x){
  values <- c(6, 12, 8, 12, 9)
  ifelse(x %in% 1:length(values), values[x], 0)
}

pd

In [ ]:
prob_dist <- data.frame(x = 1:5, 
                        prob = pd(1:5))
prob_dist

In [ ]:
prob_dist$prob_norm <- prob_dist$prob/sum(prob_dist$prob)

prob_dist

In [ ]:
barplot(prob_dist$prob,
        main = "Probability distribution",
        xlab = "State type",
        ylab = "Probability",
        names.arg = c("A","B","C","D","E"),
        col = "darkred",
        horiz = FALSE)

In [ ]:
random_walk <- function(pd, start, num_steps){
  y <- rep(0, num_steps)
  current <- start
  for (j in 1:num_steps){
    candidate <- current + sample(c(-1, 1), 1)
    prob <- pd(candidate) / pd(current)
    if (runif(1) < prob) current <- candidate
    y[j] <- current
  }
  return(y)
}

In [ ]:
out <- random_walk(pd, 2, 10000)
out

In [ ]:
data.frame(out) %>% group_by(out) %>% 
  summarize(N = n(), Prob = N / 10000) -> S

S

In [ ]:
barplot(S$Prob,
        main = "Probability distribution",
        xlab = "State type",
        ylab = "Probability",
        names.arg = c("A","B","C","D","E"),
        col = "darkred",
        horiz = FALSE)

In [ ]:
actual_distribution <- prob_dist$prob_norm
approximated_distribution <- S$Prob

merged<-cbind(actual_distribution,approximated_distribution)

merged

In [ ]:
barplot(merged,beside=TRUE,col="darkred")